## loading data/libs

In [183]:
import pandas as pd
import calendar
from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.layouts import row
from bokeh.plotting import figure

output_notebook() 

Loading BokehJS ...

In [184]:
INPUT="data/device_failure.csv" 
dataset = pd.read_csv(INPUT,index_col=[0,1],parse_dates=[0])

In [185]:
label_dset = dataset[["failure"]]


### checking devices

In [186]:
total_failures_per_device = label_dset.groupby(level=1).agg(sum)
total_failures_per_device["failure"].value_counts()

0    1062
1     106
Name: failure, dtype: int64

Each device fail at least once

~10% device failing

'only' 106 positive points

### checking Dates

In [187]:
dates = label_dset.index.get_level_values(0)
print "Range: from %s to %s" % (dates.min(), dates.max())


total_failures_per_date = label_dset.groupby(level=0).agg(sum)
print
print " n failures per date"
print str(total_failures_per_date["failure"].value_counts())
print
print "total: %i failures for %i days" % (total_failures_per_date["failure"].sum(), 
                                          total_failures_per_date[total_failures_per_date.failure >0].count())

Range: from 2015-01-01 00:00:00 to 2015-11-02 00:00:00

 n failures per date
0    228
1     54
2     19
3      2
8      1
Name: failure, dtype: int64

total: 106 failures for 76 days


In [188]:
from bokeh.plotting import figure
data =total_failures_per_date.resample("M").sum() 
test = label_dset.reset_index("device").resample("M").count()
data["n_devices"] = test["device"]
data["failure_ratio_pm"] = data["failure"] / data["n_devices"] * 1000
data.index = (calendar.month_abbr[i] for i in data.index.month)
l = Line(
    data["failure_ratio_pm"],
    title="failures per Month",
    ylabel="n_failures", 
    xlabel="month"
)
show(l)

In [189]:
weekday_dset = total_failures_per_date.copy()
weekday_dset.index = ["%i:%s" % (i,calendar.day_name[i]) for i in total_failures_per_date.index.weekday]

per_day = weekday_dset.groupby(level=0).sum()

print "failures per weekday"

per_day.sort_index()


failures per weekday


,failure
0:Monday,27
1:Tuesday,18
2:Wednesday,15
3:Thursday,22
4:Friday,12
5:Saturday,8
6:Sunday,4


- Long term trend with more failures in the past
- Less  failures over the weekend
- The absence of weekend could be explained by maintenance hapening only during workweek (hence explaing more failures on monday


### per Device description 

In [190]:
import numpy as np
dates = label_dset.swaplevel()
dd= dates.reset_index("date")["date"]
devices = pd.DataFrame({"min_date":dd.groupby(level=0).min()})
devices["max_date"] =  dd.groupby(level=0).max()
devices["n_lines"] = dd.groupby(level=0).count()
devices["n_days"] = (devices["max_date"] - devices["min_date"] ) /np.timedelta64(1, 'D') +1
devices["missing_values"] = devices["n_days"] - devices["n_lines"]

In [191]:
devices["min_date"].value_counts()

2015-01-01    1163
2015-05-06       4
2015-01-27       1
Name: min_date, dtype: int64

### checking the nb devices per month. this is better done above

In [199]:
#pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices = pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices.index = [calendar.month_abbr[i] for i in montlhy_devices.index]
montlhy_devices

,n_devices
Jan,399
Feb,46
Mar,184
Apr,112
May,72
Jun,6
Jul,15
Aug,150
Sep,38
Oct,115


### bucketing the n devices with missing day data

In [200]:
i = ( (devices["missing_values"] //20)*20).value_counts()
#i = ( (devices["missing_values"])).value_counts(bins=10)
i.index.name = "n missing days"
pd.DataFrame({"n devices":i.sort_index()})

,n devices
n missing days,
-0.0,1077
20.0,26
40.0,21
60.0,8
80.0,3
100.0,28
120.0,4
140.0,1
